In [ ]:
#Project 3

#imports
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import torch.optim as optim
import oapackage

scaler = MinMaxScaler()

In [66]:
df_onlygrad = pd.read_csv('chromdataA.csv')


df_raw = pd.read_csv('chromdataA.csv')
desired_prod_raw = df_raw.loc[(df_raw['Yield [%]'] > 80) & (df_raw['Prod [g/(l*CV)]'] > 0.4), 'Prod [g/(l*CV)]']
desired_yield_raw = df_raw.loc[(df_raw['Yield [%]'] > 80) & (df_raw['Prod [g/(l*CV)]'] > 0.4), 'Yield [%]']

print(type(df_onlygrad['Yield [%]']))

np_yields = np.array(df_onlygrad['Yield [%]']).reshape(-1,1)
scaled_np_yields = scaler.fit_transform(np_yields)
print(scaled_np_yields)



<class 'pandas.core.series.Series'>
[[0.000e+00]
 [9.523e+01]
 [9.624e+01]
 [0.000e+00]
 [6.167e+01]
 [9.723e+01]
 [9.204e+01]
 [5.775e+01]
 [0.000e+00]
 [1.000e-02]
 [8.035e+01]
 [0.000e+00]
 [5.442e+01]
 [8.933e+01]
 [2.000e-02]
 [8.814e+01]
 [4.306e+01]
 [9.972e+01]
 [9.858e+01]
 [3.494e+01]
 [1.300e-01]
 [7.696e+01]
 [1.900e-01]
 [5.626e+01]
 [3.950e+01]
 [9.995e+01]
 [9.765e+01]
 [0.000e+00]
 [0.000e+00]
 [4.200e-01]
 [7.541e+01]
 [4.000e-01]
 [9.041e+01]
 [0.000e+00]
 [9.937e+01]
 [0.000e+00]
 [9.838e+01]
 [4.531e+01]
 [5.683e+01]
 [0.000e+00]
 [9.997e+01]
 [1.730e+00]
 [0.000e+00]
 [0.000e+00]
 [5.216e+01]
 [1.480e+00]
 [9.148e+01]
 [9.070e+01]
 [9.848e+01]
 [5.183e+01]
 [3.990e+00]
 [6.650e+00]
 [0.000e+00]
 [9.100e-01]
 [9.607e+01]
 [0.000e+00]
 [1.115e+01]
 [1.300e-01]
 [1.100e-01]
 [9.982e+01]
 [0.000e+00]
 [0.000e+00]
 [0.000e+00]
 [0.000e+00]
 [0.000e+00]
 [1.210e+00]
 [0.000e+00]
 [3.712e+01]
 [6.300e+00]
 [8.620e+00]
 [4.210e+00]
 [6.270e+00]
 [0.000e+00]
 [1.303e+01]
 [

In [ ]:
yield_scaled = scaler.fit_transform(np.array(df_onlygrad['Yield [%]']).reshape(-1, 1))
prod_scaled = scaler.fit_transform(np.array(df_onlygrad['Prod [g/(l*CV)]']).reshape(-1, 1))
df_onlygrad['Yield*Prod'] =  yield_scaled * prod_scaled 
df_onlygrad = df_onlygrad.sort_values(by=['Yield*Prod'], ascending=False).head(10)

yield_data = df_onlygrad['Yield [%]']
prod_data = df_onlygrad['Prod [g/(l*CV)]']

desired_yield = df_onlygrad.loc[(df_onlygrad['Yield [%]'] > 95) & (df_onlygrad['Prod [g/(l*CV)]'] > 0.8), 'Yield [%]']
desired_prod = df_onlygrad.loc[(df_onlygrad['Yield [%]'] > 95) & (df_onlygrad['Prod [g/(l*CV)]'] > 0.8), 'Prod [g/(l*CV)]']

print("Yield: ", desired_yield)
print("Prod: ", desired_prod)

fig, axs = plt.subplots(1, 2, figsize=(15, 5))

axs[0].scatter(yield_data.values, prod_data.values)
axs[0].set_xlabel('Yield [%]')
axs[0].set_ylabel('Prod [g/(l*CV)]')
axs[0].set_title('Yield vs. Productivity')

axs[1].scatter(desired_yield_raw.values, desired_prod_raw.values)
axs[1].set_xlabel('Yield [%]')
axs[1].set_ylabel('Prod [g/(l*CV)]')
axs[1].set_title('Yield vs. Productivity')


plt.show()

In [ ]:
def get_data(filename, variables, target):    
    df_onlygrad = pd.read_csv(filename)
    
    y_yield = df_onlygrad[target].to_numpy()
    X = df_onlygrad[variables].to_numpy()

   

    X_scaled = scaler.fit_transform(X)
    y_yield_scaled = scaler.fit_transform(y_yield.reshape(-1,1))

    X_train, X_temp, y_train, y_temp = train_test_split(X_scaled, y_yield_scaled, test_size=0.3, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

    X_train = torch.tensor(X_train, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.float32)
    X_val = torch.tensor(X_val, dtype=torch.float32)
    y_val = torch.tensor(y_val, dtype=torch.float32)
    X_test = torch.tensor(X_test, dtype=torch.float32)
    y_test = torch.tensor(y_test, dtype=torch.float32)

    return X_train, y_train, X_val, y_val, X_test, y_test

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(3, 20),
            nn.ReLU(),
            nn.Linear(20, 50),
            nn.ReLU(),
            nn.Linear(50, 50),
            nn.Dropout(0.2),
            nn.ReLU(),
            nn.Linear(50, 20),
            nn.ReLU(),
            nn.Linear(20, 1),
            #nn.Sigmoid()    
        )
        
    def forward(self, x):
        return self.model(x)

In [ ]:
model1 = Model()
model2 = Model()
model3 = Model()
#loss_fn = nn.BCELoss()
loss_fn = nn.MSELoss()
optimizer1 = optim.Adam(model1.parameters())
optimizer2 = optim.Adam(model2.parameters())
model_dict = {
    "models": [model1, model2],
    "optimizers": [optimizer1, optimizer2]
}

In [ ]:
# Train the model
EPOCHS = 2000
train_losses = {}
valid_losses = {}

print("Starting training...")

filenames = ['chromdataA_onlygrad.csv','chromdataA.csv', 'chromdataB.csv']
variables = ['Bstart [%]','Bend [%]','Vload [CV]']
targets = ['Yield [%]', 'Prod [g/(l*CV)]']



for i in range(2):
    print('='*100)
    print(f"Training model for {targets[i]}")
    train_losses[targets[i]] = []
    valid_losses[targets[i]] = []
    X_train, y_train, X_val, y_val, X_test, y_test = get_data(filenames[2], variables, targets[i])

    for epoch in range(EPOCHS):
        model_dict['models'][i].train()  # Set the model to training mode
        model_dict['optimizers'][i].zero_grad()
        y_pred_train = model_dict['models'][i](X_train)
        train_loss = loss_fn(y_pred_train, y_train)
        train_loss.backward()
        model_dict['optimizers'][i].step()
        train_losses[targets[i]].append(train_loss.item())
        
        model_dict['models'][i].eval()  # Set the model to evaluation mode
        with torch.no_grad():
            y_pred_valid = model_dict['models'][i](X_val)
            valid_loss = loss_fn(y_pred_valid, y_val)
            valid_losses[targets[i]].append(valid_loss.item())
        
        # Print the loss every 500 epochs to monitor training
        if epoch % 500 == 0:
            print(f"Epoch {epoch}/{EPOCHS} - Train Loss: {train_loss.item()}, Valid Loss: {valid_loss.item()}")
        
        ## Check convergence
        #if epoch > 300:
        #    diff = train_losses[-300] - train_losses[-1]
        #    if diff > 0 and diff <= 5e-5:
        #        print(f"Model converged after {epoch} epochs\nTrain Loss: {train_loss.item()}\nDerivative: {train_losses[-300] - train_losses[-1] / len(train_losses[-300:-1])}")
        #        break

    print("Training completed for " + targets[i] + ".")
    
print("Training completed.")

In [ ]:
# Plot the loss function
fig, axs = plt.subplots(1, 2, figsize=(15, 5))

for i, target in enumerate(targets):
    axs[i].plot(train_losses[target], label='Training Loss')
    axs[i].plot(valid_losses[target], label='Validation Loss')
    axs[i].set_xlabel("Epoch")
    axs[i].set_ylabel("Loss")
    axs[i].set_title(f"Loss function during training for {target}")
    axs[i].legend()

plt.show()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15, 5))
# Loop over the three different targets and test them
for i in range(2):
    X_train, y_train, X_val, y_val, X_test, y_test = get_data(filenames[2], variables, targets[i])

    unscaled_y_test = scaler.inverse_transform(y_test)

    model_dict['models'][i].eval()  # Set the model to evaluation mode
    with torch.no_grad():
        y_pred_test = model_dict['models'][i](X_test)
        unscaled_y_pred_test = scaler.inverse_transform(y_pred_test)

    axs[i].scatter(unscaled_y_test, unscaled_y_pred_test)
    axs[i].set_xlabel(f"True {targets[i]}")
    axs[i].set_ylabel(f"Predicted {targets[i]}")
    axs[i].set_title(f"Predicted vs True {targets[i]}")

plt.show()

In [ ]:
df = pd.read_csv(filenames[2])
df.describe()

In [ ]:
Bstarts = np.linspace(24.3, 49.9, 100)
Bends = np.linspace(47.9, 47.9, 100)
Vloads = np.linspace(5.1, 5.1, 100)

#mean Bstart = 24.3
#mean Bend = 47.9
#mean Vload = 5.1

In [ ]:
Bstarts_tensor = torch.tensor(Bstarts, dtype=torch.float32)
Bends_tensor = torch.tensor(Bends, dtype=torch.float32)
Vloads_tensor = torch.tensor(Vloads, dtype=torch.float32)

X = torch.stack([Bstarts_tensor, Bends_tensor, Vloads_tensor], dim=1)

yields = model1(X).detach().numpy()
prods = model2(X).detach().numpy()

unscaled_yields = scaler.inverse_transform(yields)
unscaled_prods = scaler.inverse_transform(prods)

fig, axs = plt.subplots(1, 2, figsize=(15, 5))

axs[0].plot(Bstarts, unscaled_yields, label='Yield')
axs[0].set_xlabel("Bstart [%]")
axs[0].set_ylabel("Yield [%]")
axs[0].set_title(f"Yield vs Bstart")
axs[0].legend()

axs[1].plot(Bstarts, unscaled_prods, label='Prod')
axs[1].set_xlabel("Bstart [%]")
axs[1].set_ylabel("Prod [g/(l*CV)]")
axs[1].set_title(f"Prod vs Bstart")
axs[1].legend()

plt.show()